# 第十章 评估（下）——当不存在一个简单的正确答案时

 - [一、环境配置](#一、环境配置)
 - [二、运行问答系统获得一个复杂回答](#二、运行问答系统获得一个复杂回答)
 - [三、使用 GPT 评估回答是否正确](#三、使用-GPT-评估回答是否正确)
 - [四、给出一个标准回答，要求其评估生成回答与标准回答的差距](#四、给出一个标准回答，要求其评估生成回答与标准回答的差距)


在上一章中，了解了如何评估 LLM 模型在 **有明确正确答案** 的情况下的输出，我们可以编写一个函数来判断 LLM 输出是否正确地分类并列出产品。

然而，如果 LLM 用于生成文本，而不仅仅是分类问题的答案呢？接下来，我们将探讨如何评估这种类型<span style="color:#00ff00">（没有标准答案）</span>的 LLM 输出的方法。

## 一、环境配置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [1]:
import os
import utils_chap8_ch

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
client = OpenAI(api_key=openai_api_key)

def get_completion_from_messages(messages, 
                                 model ="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=1000):
    '''
    封装一个支持更多参数的自定义访问 gpt-4o-mini的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为gpt-4o-mini(ChatGPT)
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = client.chat.completions.create(
        model= model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message.content

## 二、运行问答系统获得一个复杂回答

In [26]:
def answer_user_msg(user_msg, product_info):
    delimiter = "####"
    system_message = f"""
    您是一家大型电子商店的客户服务助理。\
    请用友好和乐于助人的口吻回答问题，提供简洁明了的答案。\
    确保向用户提出相关的后续问题。
    """
    messages =  [  
    {'role':'system', 'content': system_message},   
    {'role':'user', 'content': f"{delimiter}{user_msg}{delimiter}"},  
    {'role':'assistant', 'content': f"相关产品信息:\n{product_info}"},   
    ] 
    response = get_completion_from_messages(messages)
    return response

In [27]:
# 用户消息
customer_msg = f"""
告诉我有关 smartx pro phone 和 fotosnap camera, the dslr one 的信息。
另外，你们这有什么电视？"""

# 从问题中抽取商品名
products_by_category = utils_chap8_ch.find_category_and_product(customer_msg, utils_chap8_ch.get_products_and_category())
# 将商品名转化为列表
category_and_product_list = utils_chap8_ch.read_string_to_list(products_by_category)
# 查找商品对应的信息
product_info = utils_chap8_ch.generate_output_string(category_and_product_list)
# 由信息生成回答
assistant_answer = answer_user_msg(user_msg=customer_msg, product_info=product_info)

In [28]:
print(assistant_answer) 

关于您提到的产品：

**SmartX ProPhone**
- **特点**：6.1英寸显示屏，128GB存储，12MP双摄像头，支持5G。
- **评分**：4.6
- **价格**：899.99元
- **描述**：一款拥有先进摄像功能的强大智能手机。

**FotoSnap DSLR Camera**
- **特点**：24.2MP传感器，1080p视频，3英寸LCD，支持可更换镜头。
- **评分**：4.7
- **价格**：599.99元
- **描述**：使用这款多功能的单反相机，捕捉惊艳的照片和视频。

关于电视，我们有几款不错的选择：

1. **CineView 4K TV**
   - **尺寸**：55英寸
   - **分辨率**：4K
   - **价格**：599.99元
   - **描述**：色彩鲜艳、智能功能丰富的4K电视。

2. **CineView 8K TV**
   - **尺寸**：65英寸
   - **分辨率**：8K
   - **价格**：2999.99元
   - **描述**：通过这款惊艳的8K电视，体验未来。

3. **CineView OLED TV**
   - **尺寸**：55英寸
   - **分辨率**：4K
   - **价格**：1499.99元
   - **描述**：体验真正的五彩斑斓。

您对哪款电视或手机感兴趣呢？或者您还有其他问题吗？


## 三、使用 GPT 评估回答是否正确

我们希望您能从中学到一个设计模式，即当您可以指定一个评估 LLM 输出的标准列表时，您实际上可以<span style="color:red">使用另一个 API 调用</span>来评估您的第一个 LLM 输出。

In [29]:
# 问题、上下文
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': product_info
}

In [30]:
def eval_with_rubric(test_set, assistant_answer):
    """
    使用 GPT API 评估生成的回答

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    
    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    # 人设
    system_message = """\
    你是一位助理，通过查看客户服务代理使用的上下文来评估客户服务代理回答用户问题的情况。
    """

    # 具体指令
    user_message = f"""\
    你正在根据代理使用的上下文评估对问题的提交答案。以下是数据：
    [开始]
    ************
    [用户问题]: {cust_msg}
    ************
    [使用的上下文]: {context}
    ************
    [客户代理的回答]: {completion}
    ************
    [结束]

    请将客户代理的回答的事实内容与上下文进行比较，忽略样式、语法或标点符号上的差异。

    请回答以下问题：
    助手的回应是否只基于所提供的上下文？（是或否）
    回答中是否包含上下文中未提供的信息？（是或否）
    回答与上下文之间是否存在任何不一致之处？（是或否）
    计算用户提出了多少个问题。（输出一个数字）
    对于用户提出的每个问题，是否有相应的回答？请按照以下格式进行输出，需要输出具体的用户问题：
    <用户提出的问题1>：（是或否）
    <用户提出的问题2>：（是或否）
    ...
    <用户提出的问题N>：（是或否）
    在提出的问题数量中，有多少个问题在回答中得到了回应？（输出一个数字）
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [31]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

助手的回应是否只基于所提供的上下文？（是）
回答中是否包含上下文中未提供的信息？（否）
回答与上下文之间是否存在任何不一致之处？（否）
计算用户提出了多少个问题。（2）
对于用户提出的每个问题，是否有相应的回答？请按照以下格式进行输出：
告诉我有关 smartx pro phone 和 fotosnap camera, the dslr one 的信息：（是）
另外，你们这有什么电视？：（是）
在提出的问题数量中，有多少个问题在回答中得到了回应？（2）


## 四、给出一个标准回答，要求其评估生成回答与标准回答的差距

在经典的自然语言处理技术中，有一些传统的度量标准用于衡量 LLM 输出与人类专家编写的输出的相似度。例如，BLUE 分数可用于衡量两段文本的相似程度。

实际上有一种更好的方法，即使用 Prompt。您可以指定 Prompt，<span style="color:red">使用 Prompt</span> 来比较由 LLM 自动生成的客户服务代理响应与人工理想响应的匹配程度。

In [32]:
test_set_ideal = {
    'customer_msg': 
    """
    告诉我有关 smartx pro phone 和 fotosnap camera, the dslr one 的信息。\n
    另外，你们这有什么电视？
    """,
    'ideal_answer':
    """
    SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。
    FotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。
    我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年。
    """
}

In [33]:
def eval_vs_ideal(test_set, assistant_answer):
    """
    评估回复是否与理想答案匹配

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    system_message = """\
    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。
    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 
    """

    user_message = f"""\
    您正在比较一个给定问题的提交答案和专家答案。数据如下:
    [开始]
    ************
    [问题]: {cust_msg}
    ************
    [专家答案]: {ideal}
    ************
    [提交答案]: {completion}
    ************
    [结束]

    比较提交答案的事实内容与专家答案。忽略样式、语法或标点符号上的差异。
    提交的答案可能是专家答案的子集、超集，或者与之冲突。确定适用的情况，并通过选择以下选项之一回答问题：
    （A）提交的答案是专家答案的子集，并且与之完全一致。
    （B）提交的答案是专家答案的超集，并且与之完全一致。
    （C）提交的答案包含与专家答案完全相同的细节。
    （D）提交的答案与专家答案存在分歧。
    （E）答案存在差异，但从事实的角度来看这些差异并不重要。
    选项：ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

这个评分标准来自于 <span style="color:red">OpenAI 开源评估框架</span>，这是一个非常棒的框架，其中包含了许多评估方法，既有 OpenAI 开发人员的贡献，也有更广泛的开源社区的贡献。

在这个评分标准中，我们要求 LLM 针对提交答案与专家答案进行信息内容的比较，并忽略其风格、语法和标点符号等方面的差异，但关键是我们要求它进行比较，并输出从A到E的分数，具体取决于提交的答案是否是专家答案的子集、超集或完全一致，这可能意味着它虚构或编造了一些额外的事实。

LLM 将选择其中最合适的描述。


In [34]:
print(assistant_answer)

关于您提到的产品：

**SmartX ProPhone**
- **特点**：6.1英寸显示屏，128GB存储，12MP双摄像头，支持5G。
- **评分**：4.6
- **价格**：899.99元
- **描述**：一款拥有先进摄像功能的强大智能手机。

**FotoSnap DSLR Camera**
- **特点**：24.2MP传感器，1080p视频，3英寸LCD，支持可更换镜头。
- **评分**：4.7
- **价格**：599.99元
- **描述**：使用这款多功能的单反相机，捕捉惊艳的照片和视频。

关于电视，我们有几款不错的选择：

1. **CineView 4K TV**
   - **尺寸**：55英寸
   - **分辨率**：4K
   - **价格**：599.99元
   - **描述**：色彩鲜艳、智能功能丰富的4K电视。

2. **CineView 8K TV**
   - **尺寸**：65英寸
   - **分辨率**：8K
   - **价格**：2999.99元
   - **描述**：通过这款惊艳的8K电视，体验未来。

3. **CineView OLED TV**
   - **尺寸**：55英寸
   - **分辨率**：4K
   - **价格**：1499.99元
   - **描述**：体验真正的五彩斑斓。

您对哪款电视或手机感兴趣呢？或者您还有其他问题吗？


<span style="color:#00ff00">
对于我们的生成回答，gpt-4o-mini 判断生成内容与专家内容完全一致，即C
</span>

In [ ]:
eval_vs_ideal(test_set_ideal, assistant_answer)
# 对于该生成回答，GPT 判断生成内容是标准回答的一个子集，即A

'C'

In [36]:
assistant_answer_2 = """生活就像一袋巧克力。"""

In [37]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)
# 对于明显异常答案，GPT 判断为不一致

'D'

希望您从本章中学到两个设计模式。<span style="color:#00ff00">（核心：使用另一个LLM调用来评估当前LLM的输出）</span>

1. 即使没有专家提供的理想答案，只要能制定一个评估标准，就可以使用一个 LLM 来评估另一个 LLM 的输出。

2. 如果您可以提供一个专家提供的理想答案，那么可以帮助您的 LLM 更好地比较特定助手输出是否与专家提供的理想答案相似。

希望这可以帮助您评估 LLM 系统的输出，以便在开发期间持续监测系统的性能，并使用这些工具不断评估和改进系统的性能。